In [2]:
import duckdb
import hashlib
import os
import json

In [3]:
def getmd5(file_path, blocksize=2**20):
    m = hashlib.md5()
    with open( os.path.join(file_path) , "rb" ) as f:
        while True:
            buf = f.read(blocksize)
            if not buf:
                break
            m.update( buf )
    return m.hexdigest()

In [4]:
# filePath = '/Users/yoonsikbyun/Documents/minikube_mnt/mlstudio/interface/sample/total_bank_data.csv'
# filePath = '/Users/yoonsikbyun/Documents/minikube_mnt/mlstudio/interface/sample/bank.csv'
# parquetFile = '/Users/yoonsikbyun/Documents/minikube_mnt/mlstudio/interface/sample/bank.parquet'
# parquetFile = '/Users/yoonsikbyun/Documents/minikube_mnt/mlstudio/interface/sample/total_bank_data.parquet'

filePath = 'E:/minikube_mnt/mlstudio/interface/sample/kaggle/total_bank_data.csv'
# filePath = 'E:/minikube_mnt/mlstudio/interface/sample/kaggle/bank.csv'
# parquetFile = 'E:/minikube_mnt/mlstudio/interface/sample/kaggle/bank.parquet'
parquetFile = 'E:/minikube_mnt/mlstudio/interface/sample/kaggle/total_bank_data.parquet'

In [5]:
fileName = os.path.basename(filePath)
dirName = os.path.dirname(filePath)
fileName = os.path.join(dirName, fileName + '.parquet')


In [10]:

def writeFileMetaInfo(file_path : str, md5 : str = None, summarize : dict = None) :
    fileName = os.path.basename(file_path)
    fileName, ext = os.path.splitext(fileName)
    dirName = os.path.dirname(file_path)
    metaFileName = os.path.join(dirName, fileName + '.meta')
    metaDict = {}

    if os.path.exists(metaFileName):
        with open(metaFileName, 'r') as fp :
            metaDict = json.load(fp)
            metaDict['file_path'] = file_path
            metaDict['extension'] = ext

    metaDict['file_path'] = file_path
    if md5: metaDict['md5'] = md5
    if summarize : metaDict['summarize'] = summarize

    with open(metaFileName, 'w') as fp:
        json.dump(metaDict, fp)


def getFileMetaInfo(file_path : str) :
    fileName = os.path.basename(file_path)
    fileName, _ = os.path.splitext(fileName)
    dirName = os.path.dirname(file_path)
    metaFileName = os.path.join(dirName, fileName + '.meta')

    if os.path.exists(metaFileName):
        with open(metaFileName, 'r') as fp :
            return json.load(fp)

    return None

In [11]:
md5 = getmd5(file_path=filePath)
print(md5)
writeFileMetaInfo(file_path=filePath, md5=md5)
rtn = getFileMetaInfo(file_path=filePath)
print(rtn)



0bde8b948ca1e78c31e19900b9729267
{'md5': '0bde8b948ca1e78c31e19900b9729267', 'file_path': 'E:/minikube_mnt/mlstudio/interface/sample/kaggle/total_bank_data.csv', 'extension': '.csv'}


In [12]:
def convertCsvIntoParquet(file_path : str) :
    fileName = os.path.basename(file_path)
    fileName, _ = os.path.splitext(fileName)
    dirName = os.path.dirname(file_path)
    parquetFile = os.path.join(dirName, fileName + '.parquet')

    sql = f"COPY (SELECT * FROM read_csv('{file_path}',AUTO_DETECT=TRUE)) TO '{parquetFile}' (FORMAT 'PARQUET', CODEC 'ZSTD')"
    duckdb.sql(sql)

    md5 = getmd5(file_path=parquetFile)
    writeFileMetaInfo(file_path=parquetFile, md5=md5)

In [14]:
convertCsvIntoParquet(file_path=filePath)
rtn = getFileMetaInfo(file_path=parquetFile)
print(rtn)

{'md5': '12aea6e5e2d813da5559abed4da5bd35', 'file_path': 'E:/minikube_mnt/mlstudio/interface/sample/kaggle\\total_bank_data.parquet', 'extension': '.parquet'}


In [57]:
sql = f"SELECT * FROM read_csv('{filePath}') LIMIT 100"
print(sql)
duckdb.sql(sql)

SELECT * FROM read_csv('/Users/yoonsikbyun/Documents/minikube_mnt/mlstudio/interface/sample/total_bank_data.csv') LIMIT 100


┌─────────┬──────────┬─────────┬──────────┬───┬────────────────────┬────────┬───────────┬────────────────────┐
│ ABCUBKR │ ABCUOTHR │ ABCXBKR │ ABCXOTHR │ … │        UCR         │ UCSCR  │ UNINCFORR │      VOLIABR       │
│ double  │  double  │ double  │  double  │   │       double       │ double │  double   │       double       │
├─────────┼──────────┼─────────┼──────────┼───┼────────────────────┼────────┼───────────┼────────────────────┤
│     0.0 │      0.0 │     0.0 │      0.0 │ … │  7.938524701350142 │    0.0 │       0.0 │  8.609526048778724 │
│     0.0 │      0.0 │     0.0 │      0.0 │ … │ 12.942877100391945 │    0.0 │       0.0 │  7.832570142419186 │
│     0.0 │      0.0 │     0.0 │      0.0 │ … │ 10.463096093937216 │    0.0 │       0.0 │  4.849628564581836 │
│     0.0 │      0.0 │     0.0 │      0.0 │ … │ 14.322079797797436 │    0.0 │       0.0 │  6.151531564060901 │
│     0.0 │      0.0 │     0.0 │      0.0 │ … │ 22.592049158895435 │    0.0 │       0.0 │  5.047434182756467 │
│

In [58]:
sql = f"SELECT * FROM read_parquet('{parquetFile}') LIMIT 100"
print(sql)
duckdb.sql(sql)

SELECT * FROM read_parquet('/Users/yoonsikbyun/Documents/minikube_mnt/mlstudio/interface/sample/total_bank_data.parquet') LIMIT 100


┌─────────┬──────────┬─────────┬──────────┬───┬────────────────────┬────────┬───────────┬────────────────────┐
│ ABCUBKR │ ABCUOTHR │ ABCXBKR │ ABCXOTHR │ … │        UCR         │ UCSCR  │ UNINCFORR │      VOLIABR       │
│ double  │  double  │ double  │  double  │   │       double       │ double │  double   │       double       │
├─────────┼──────────┼─────────┼──────────┼───┼────────────────────┼────────┼───────────┼────────────────────┤
│     0.0 │      0.0 │     0.0 │      0.0 │ … │  7.938524701350142 │    0.0 │       0.0 │  8.609526048778724 │
│     0.0 │      0.0 │     0.0 │      0.0 │ … │ 12.942877100391945 │    0.0 │       0.0 │  7.832570142419186 │
│     0.0 │      0.0 │     0.0 │      0.0 │ … │ 10.463096093937216 │    0.0 │       0.0 │  4.849628564581836 │
│     0.0 │      0.0 │     0.0 │      0.0 │ … │ 14.322079797797436 │    0.0 │       0.0 │  6.151531564060901 │
│     0.0 │      0.0 │     0.0 │      0.0 │ … │ 22.592049158895435 │    0.0 │       0.0 │  5.047434182756467 │
│